# Setup

In [28]:
import numpy as np
import pandas as pd
import networkx as nx

import seaborn as sns
sns.set_style("ticks")
sns.set_context('notebook')
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# import graph.data
# import graph.network
# import graph.temporal
# import graph.utils

from graph.data import *
from graph.network import *
from graph.temporal import *
from graph.utils import *

In [29]:
# import importlib

# importlib.reload(graph.data)
# importlib.reload(graph.network)
# importlib.reload(graph.temporal)
# importlib.reload(graph.utils)

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
phonelab_to_db()

0 : 0031e78c899fbc79f2d44a0db30f30a8acdf8767
1 : 0191c805a5f1ef0e10b046658fdbe026e33ca2b1
2 : 023735506ab66eac2e18a14f2f391447d70c6e5b
3 : 05bf808c71cabb62c7f436cbee1860f980683b1a
4 : 067fd335de0e2336d9172e0438e688e67f7c822e
5 : 087d92ff25c1a6a29a99a53cbb335b65d7ce9dbc
6 : 094e84e4907cbd1a9889f5d5bd2588538daae853
7 : 09db1f59f45bfdb182f1ea4b3e947b86e5773d0e
8 : 09f9a05e8775d4df75cfe9bd546257272f7ec66e
9 : 0b94bcb4cfea1bc9598c9510110f10bf18cd260e
10 : 0bc6c38a5f46059fa6035732bb2e03f8800d28e8
11 : 0c037a6e55da4e024d9e64d97114c642695c5434
12 : 0c913e6f38937b895d3c25a79fc09aff9f9f407a
13 : 0d96b3d6c0a4fabd3dcd31cdf2513b60a4616556
14 : 0ee9cead2e2a3a58a316dc27571476e8973ff944
15 : 0f73f649f1e0bb371f1fdcadf86f02567670315a
16 : 1452809c9c0f670bd138f47961cb6814b3891848
17 : 16279792b28ca97c9c224d27cef263d55b97f8ae
18 : 1691ee6e8576a4015b86e27b3e608522c4e0d47d
19 : 178e5ffd59070a23760ce38cd8c051f3fd569d97
20 : 17ed4022f242778280c10362077816b61fd37d42
21 : 1924a6a39f09d992514925466c8844462377205